In [1]:
pd.options.display.max_rows = 9999
pd.options.display.max_columns = 9999

import sys
import missingno as msno
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
from PIL import Image
from functools import reduce

%matplotlib inline
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

In [2]:
df_credit_2020 = pd.read_excel("2020_credit_rating.xlsx")
df_credit_2020.head()

,Unnamed: 0,평가회사,평가대상회사,평가대상종류,공시일자,평가등급,등급감시,등급전망
0,1,한국기업평가,삼성엔지니어링(주),본평가,2020-01-02,A-,NaN,안정적
1,2,한국기업평가,삼화페인트공업(주),수시평가,2020-01-02,BBB+,NaN,안정적
2,3,나이스신용평가,예금보험공사,본평가,2020-01-08,AAA,NaN,안정적
3,4,한국기업평가,에스케이증권(주),본평가,2020-01-09,A,NaN,안정적
4,5,한국기업평가,우리종합금융(주),본평가 (정정),2020-01-09,A,NaN,안정적


In [3]:
credit_dlist={"D":0,"C-":1,"C":2,"C+":3,"CC-":4,"CC":5,"CC+":6,"CCC-":7,"CCC":8,"CCC+":9,
              "B-":10,"B":11,"B+":12,"BB-":13,"BB":14,"BB+":15,"BBB-":16,"BBB":17,"BBB+":18,
              "A-":19,"A":20,"A+":21,"AA-":22,"AA":23,"AA+":24,"AAA-":25,"AAA":26,"AAA+":27}


def screen_name(df):
    df = df.rename(columns = {"평가대상회사":"기업명"})
    df = df[["기업명","공시일자","평가등급"]]
    df["공시일자"] = df["공시일자"].apply(lambda x: int(str(x)[:4]))
    df["평가등급"][df["평가등급"] == "취소"] = np.nan
    df["기업명"] = df["기업명"].apply(lambda x: str(x).replace("(주)",""))
    df = df.dropna()
    df = df.drop_duplicates(['기업명'],keep="last").reset_index(drop=True)
    df['평가등급'] = df['평가등급'].apply(credit_dlist.get)
    return df

In [4]:
df_credit_2017 = screen_name(pd.read_excel("2017_credit_rating.xlsx"))
df_credit_2018 = screen_name(pd.read_excel("2018_credit_rating.xlsx"))
df_credit_2019 = screen_name(pd.read_excel("2019_credit_rating.xlsx"))
df_credit_2020 = screen_name(pd.read_excel("2020_credit_rating.xlsx"))

In [5]:
df_credit_2020.tail()

,기업명,공시일자,평가등급
206,에스케이네트웍스,2020,22
207,농협금융지주,2020,26
208,키움예스저축은행,2020,18
209,키움저축은행,2020,19
210,한전KPS,2020,23


In [6]:
df_2013_2016_credit = pd.read_excel("2013-2016_credit_rating.xlsx")
df_2013_2016_credit.columns = ["nnamed", "평가회사","기업명","평가","평가대상종류","공시일자","평가등급","등급감시","등급전망"]
df_2013_2016_credit["공시일자"] = df_2013_2016_credit["공시일자"].apply(lambda x: int(str(x)[:4]))

In [7]:
df_2013_2016_credit.head()

,nnamed,평가회사,기업명,평가,평가대상종류,공시일자,평가등급,등급감시,등급전망
0,2795,나이스신용평가,대우조선해양(주),기업신용평가,수시평가,2016,B+,NaN,부정적
1,2794,나이스신용평가,(주)엘에스네트웍스,기업신용평가,수시평가,2016,A-,NaN,부정적
2,2793,나이스신용평가,동국산업(주),기업신용평가,본평가,2016,BBB+,NaN,안정적
3,2792,한국신용평가,엘에스오토모티브(주),기업신용평가,본평가,2016,A-,NaN,안정적
4,2791,나이스신용평가,(주)두산캐피탈,기업신용평가,수시평가,2016,BBB-,↑,NaN


In [8]:
df_credit_2013 = screen_name(df_2013_2016_credit[df_2013_2016_credit["공시일자"] == 2013])
df_credit_2014 = screen_name(df_2013_2016_credit[df_2013_2016_credit["공시일자"] == 2014])
df_credit_2015 = screen_name(df_2013_2016_credit[df_2013_2016_credit["공시일자"] == 2015])
df_credit_2016 = screen_name(df_2013_2016_credit[df_2013_2016_credit["공시일자"] == 2016])

In [9]:
df_a_1 = pd.merge(df_credit_2014, df_credit_2015, on = "기업명",how = "inner").dropna()
df_a_1["Point"] = df_a_1["평가등급_x"] - df_a_1["평가등급_y"]
df_a_1["Point"][df_a_1["Point"].values != 0] =1
df_a_1 = df_a_1[["기업명","Point"]]
df_a_2 = pd.merge(df_a_1, df_credit_2015, on="기업명", how="inner")

In [10]:
df_b_1 = pd.merge(df_credit_2015, df_credit_2016, on = "기업명",how = "inner").dropna()
df_b_1["Point"] = df_b_1["평가등급_x"] - df_b_1["평가등급_y"]
df_b_1["Point"][df_b_1["Point"].values != 0] =1
df_b_1 = df_b_1[["기업명","Point"]]
df_b_2 = pd.merge(df_b_1, df_credit_2016, on="기업명", how="inner")

In [11]:
df_b_2

,기업명,Point,공시일자,평가등급
0,엠에스오토텍,0.0,2016,14.0
1,대한전선,0.0,2016,15.0
2,우룡,0.0,2016,13.0
3,우리종합금융,0.0,2016,19.0
4,중국농업은행서울지점,0.0,2016,26.0
5,한국철강,0.0,2016,20.0
6,에스케이건설,0.0,2016,19.0
7,오뚜기,0.0,2016,23.0
8,한국투자저축은행,0.0,2016,20.0
9,엘아이지,0.0,2016,21.0


In [12]:
df_c_1 = pd.merge(df_credit_2016, df_credit_2017, on = "기업명",how = "inner").dropna()
df_c_1["Point"] = df_c_1["평가등급_x"] - df_c_1["평가등급_y"]
df_c_1["Point"][df_c_1["Point"].values != 0] =1
df_c_1 = df_c_1[["기업명","Point"]]
df_c_2 = pd.merge(df_c_1, df_credit_2017, on="기업명", how="inner")

In [13]:
df_credit_2017

,기업명,공시일자,평가등급
0,블루에이치제일차,2017,24.0
1,플렉스컴,2017,0.0
2,네이처브리지,2017,11.0
3,한화시스템,2017,22.0
4,에스엠면세점,2017,10.0
5,하나투어,2017,20.0
6,대구도시공사,2017,24.0
7,서진오토모티브,2017,16.0
8,에스케이머티리얼즈,2017,21.0
9,에스케이인천석유화학,2017,21.0


In [14]:
df_d_1 = pd.merge(df_credit_2017, df_credit_2018, on = "기업명",how = "inner").dropna()
df_d_1["Point"] = df_d_1["평가등급_x"] - df_d_1["평가등급_y"]
df_d_1["Point"][df_d_1["Point"].values != 0] =1
df_d_1 = df_d_1[["기업명","Point"]]
df_d_2 = pd.merge(df_d_1, df_credit_2018, on="기업명", how="inner")

In [15]:
df_e_1 = pd.merge(df_credit_2018, df_credit_2019, on = "기업명",how = "inner").dropna()
df_e_1["Point"] = df_e_1["평가등급_x"] - df_e_1["평가등급_y"]
df_e_1["Point"][df_e_1["Point"].values != 0] =1
df_e_1 = df_e_1[["기업명","Point"]]
df_e_2 = pd.merge(df_e_1, df_credit_2019, on="기업명", how="inner")

In [16]:
df_f_1 = pd.merge(df_credit_2019, df_credit_2020, on = "기업명",how = "inner").dropna()
df_f_1["Point"] = df_f_1["평가등급_x"] - df_f_1["평가등급_y"]
df_f_1["Point"][df_f_1["Point"].values != 0] =1
df_f_1 = df_f_1[["기업명","Point"]]
df_f_2 = pd.merge(df_f_1, df_credit_2020, on="기업명", how="inner")

In [17]:
df_credit_2015 = df_a_2
df_credit_2016 = df_b_2
df_credit_2017 = df_c_2
df_credit_2018 = df_d_2
df_credit_2019 = df_e_2
df_credit_2020 = df_f_2

In [18]:
df_credit_2020

,기업명,Point,공시일자,평가등급
0,에이치에스디엔진,0,2020,15
1,엘지이노텍,0,2020,22
2,송원산업,0,2020,19
3,엘아이지넥스원,0,2020,22
4,한국수자원공사,0,2020,26
5,한국도로공사,0,2020,26
6,지에스,0,2020,23
7,소프트웨어공제조합,0,2020,23
8,한국스탠다드차타드은행,0,2020,26
9,녹십자,0,2020,22


In [19]:
df_esg = pd.read_csv("2011-2019-ESG.tsv", sep='\t')
df_esg.head()

,NO,기업명,기업코드,ESG등급,지배구조,사회,환경,평가년도,비고
0,6567,AJ네트웍스,95570,B,B,B,C이하,2019,NaN
1,6566,AJ렌터카,68400,C이하,C,B,B,2019,NaN
2,6565,AK홀딩스,6840,B,B,B,C이하,2019,NaN
3,6564,AP시스템,265520,C이하,B,C이하,C이하,2019,NaN
4,6563,BGF,27410,B,B,A,C이하,2019,NaN


In [20]:
df_esg = df_esg.drop(['NO', '비고'], axis=1).dropna()
df_esg['ESG등급'] = df_esg['ESG등급'].apply({"S":70.0,"A+":60.0,"A":50.0,"B+":40.0,"B":30.0,"B이하":30.0,"C":20.0,"C이하":20.0,"D":10.0}.get) 
df_esg['지배구조'] = df_esg['지배구조'].apply({"S":70.0,"A+":60.0,"A":50.0,"B+":40.0,"B":30.0,"B이하":30.0,"C":20.0,"C이하":20.0,"D":10.0}.get) 
df_esg['사회']    = df_esg['사회'].apply({"S":70.0,"A+":60.0,"A":50.0,"B+":40.0,"B":30.0,"B이하":30.0,"C":20.0,"C이하":20.0,"D":10.0}.get) 
df_esg['환경']    = df_esg['환경'].apply({"S":70.0,"A+":60.0,"A":50.0,"B+":40.0,"B":30.0,"B이하":30.0,"C":20.0,"C이하":20.0,"D":10.0}.get) 
df_esg["기업코드"] = df_esg["기업코드"].apply(lambda x: str(x))
dflist = list(df_esg["기업코드"].values)
for i in range(len(df_esg)):
    if len(dflist[i]) < 3:
        dflist[i] = "0000" + dflist[i]
    elif len(dflist[i]) < 4:
        dflist[i] ="000" + dflist[i]
    elif len(dflist[i]) < 5:
        dflist[i] ="00" + dflist[i]
    elif len(dflist[i]) < 6:
        dflist[i] = "0" + dflist[i]
df_esg["기업코드"] = dflist 

In [21]:
df_esg.tail()

,기업명,기업코드,ESG등급,지배구조,사회,환경,평가년도
6555,화천기계,010660,30.0,30.0,30.0,30.0,2011
6559,효성,004800,30.0,40.0,30.0,40.0,2011
6561,후성,093370,30.0,20.0,30.0,30.0,2011
6562,휴니드테크놀러지스,005870,40.0,20.0,40.0,30.0,2011
6564,휴켐스,069260,30.0,40.0,30.0,40.0,2011


In [22]:
mod = sys.modules[__name__]

for i in range(2011,2020):
    setattr(mod, 'df_esg_{}'.format(i),df_esg[df_esg["평가년도"]==i])

In [23]:
df_esg_2019.tail()

,기업명,기업코드,ESG등급,지배구조,사회,환경,평가년도
918,휴온스글로벌,084110,30.0,40.0,40.0,20.0,2019
919,휴젤,145020,20.0,30.0,20.0,20.0,2019
920,휴켐스,069260,30.0,30.0,40.0,30.0,2019
922,흥국화재,000540,30.0,30.0,30.0,20.0,2019
923,흥아해운,003280,20.0,30.0,20.0,20.0,2019


In [24]:
df2 = pd.read_csv("marcap-2011.csv")
df2.head()

,Code,Name,Close,Changes,ChagesRatio,Volume,Amount,Open,High,Low,Marcap,MarcapRatio,Stocks,ForeignShares,ForeignRatio,Rank,Date
0,005930,삼성전자,958000,9000,1.0,265562,254246714088,955000,966000,950000,141112764846000,11.27,147299337,74518914.0,50.59,1,2011-01-03
1,005490,POSCO,486000,-1000,-0.2,170542,82998761000,487000,489500,482000,42372801810000,3.38,87186835,43377735.0,49.75,2,2011-01-03
2,005380,현대차,177000,3500,2.0,556135,97468816500,174000,177000,173500,38988936783000,3.11,220276479,94245489.0,42.79,3,2011-01-03
3,009540,현대중공업,435000,-8000,-1.8,256635,111611214500,443000,443500,431000,33060000000000,2.64,76000000,15350207.0,20.20,4,2011-01-03
4,012330,현대모비스,294500,10000,3.5,212902,61730122000,287500,294500,285000,28667767653500,2.29,97343863,46134411.0,47.39,5,2011-01-03


In [25]:
file_list_a = ["marcap-2011.csv","marcap-2012.csv","marcap-2013.csv","marcap-2014.csv","marcap-2015.csv",
               "marcap-2016.csv","marcap-2017.csv","marcap-2018.csv","marcap-2019.csv"]

def screen_data(df):
    df = df[["Code","Name","Date","Marcap"]]
    df["Date"] = df["Date"].apply(lambda x: int(str(x).replace("-","")))
    df = df.drop_duplicates(['Name'],keep="last")
    df = df.rename(columns = {"Code":"기업코드"})
    return df
    
for i in range(2011,2020):
    setattr(mod, 'df_market_{}'.format(i),screen_data(pd.read_csv(file_list_a[i-2011])))

In [26]:
df_market_2019.tail()

,기업코드,Name,Date,Marcap
590006,245450,씨앤에스링크,20191230,2701731600
590007,276240,엘리비젼,20191230,2587200000
590008,001529,동양3우B,20191230,2552590900
590009,238500,로보쓰리,20191230,2224028850
590010,208890,에듀파트너,20191230,1935738000


In [27]:
df3 = pd.read_excel("영업활동현금흐름.xlsx")
df3.tail()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
2212,950130,엑세스바이오,12,천원,-422239.0,-5730652.0,-5445311.0,2553507.0,2967795.0,-2633921.0,1587126.0,1845660.0,-2351654.0,1915059.0,NaN,NaN
2213,950140,잉글우드랩,12,천원,12115959.0,-9368274.0,-8980775.0,-3506442.0,-471765.0,6338815.0,NaN,NaN,NaN,NaN,NaN,NaN
2214,950160,코오롱티슈진,12,천원,-43856498.0,-36048325.0,-38815829.0,-3498024.0,-5610900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2215,950170,JTC,02,천원,NaN,13138388.0,3706136.0,46183777.0,9076766.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2216,950180,SNK,07,천원,32821083.0,22198273.0,25146954.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_f = ["영업활동현금흐름.xlsx","EBITDA.xlsx","CAPEX.xlsx","당기순이익.xlsx","총차입부채.xlsx","자산총계.xlsx","매출액.xlsx","EBIT.xlsx",
       "차입부채_EBITDA.xlsx","부채비율.xlsx","차입금의존도.xlsx"]

def change_columns(df):
    columns = ["종목코드","기업명","결산월","단위","2019","2018","2017","2016","2015","2014","2013","2012","2011","2010","2009","2008"]
    df.columns = columns
    df = df.rename(columns = {"종목코드":"기업코드"})
    return df

for i in range(1,12):
    setattr(mod, 'df_balance_{}'.format(i), change_columns(pd.read_excel(df_f[i-1])[9:]))
    

In [29]:
df_balance_3[df_balance_3.values == "N/A(IFRS)"] = np.nan
df_balance_10[df_balance_10.values == "완전잠식"] = np.nan
for i in range(4,16):
    df_balance_10.iloc[:,i] = df_balance_10.iloc[:,i].apply(lambda x: float(x))

In [137]:
len(df_balance_11.dropna())

1660

In [30]:
df_balance_10.iloc[:,4:]

,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008
9,24.98,24.61,23.67,28.34,37.84,36.28,39.14,42.86,47.82,53.51,47.93,51.67
10,309.21,121.62,139.80,137.27,132.57,198.94,104.02,96.27,93.90,116.86,124.00,100.23
11,84.85,83.67,82.23,91.50,98.90,101.00,103.79,115.08,114.76,110.01,107.38,93.31
12,76.87,65.53,68.88,71.26,70.98,69.00,56.50,65.27,45.22,57.12,64.51,58.79
13,216.58,194.92,188.10,163.26,161.25,157.72,156.89,149.72,151.06,132.47,95.40,177.86
14,28.32,31.61,30.40,36.71,37.86,26.66,23.26,18.08,17.67,17.44,17.27,20.44
15,149.23,151.33,127.77,101.56,89.76,99.19,105.00,57.62,58.75,77.22,84.22,30.90
16,290.48,264.06,257.66,229.63,266.08,267.20,244.19,229.15,246.09,241.92,172.99,149.73
17,327.74,304.20,278.37,262.81,275.96,252.37,244.07,345.12,80.46,72.85,407.56,441.03
18,33.86,32.55,30.31,26.34,29.96,33.99,27.85,27.62,137.15,150.85,243.14,265.98


In [31]:
def merge_code(dfa,dfb):
    dfb = dfb[["기업코드","기업명"]][dfb["기업명"].isin(dfa["기업명"])]
    dfa = pd.merge(dfa,dfb, on="기업명",how="inner")
    return dfa

In [32]:
df_credit_2020.tail()

,기업명,Point,공시일자,평가등급
191,유안타증권,0,2020,21
192,동국제강,0,2020,16
193,한국해외인프라도시개발지원공사,0,2020,26
194,중국은행서울지점,0,2020,26
195,동국산업,0,2020,18


In [33]:
list_credit = [df_credit_2013,df_credit_2014,df_credit_2015,df_credit_2016,df_credit_2017,
             df_credit_2018,df_credit_2019]

list_esg = [df_esg_2011,df_esg_2012,df_esg_2013,df_esg_2014,df_esg_2015,df_esg_2016,
            df_esg_2017,df_esg_2018,df_esg_2019]

list_marcap = [df_market_2011,df_market_2012,df_market_2013,df_market_2014,df_market_2015,df_market_2016,
               df_market_2017,df_market_2018,df_market_2019]

list_balance = [df_balance_1,df_balance_2,df_balance_3,df_balance_4,df_balance_5,
               df_balance_6,df_balance_7,df_balance_8,df_balance_9,df_balance_10,df_balance_11]

In [34]:
for i in range(2013,2020):
    setattr(mod, 'df_rating_{}'.format(i), merge_code(list_credit[i-2013],list_esg[i-2013]))

for k in range(2011, 2020):
    for i in range(1,12):
        setattr(mod, 'df_{}_{}'.format(k,i), pd.DataFrame(list_balance[i-1].iloc[:,[0,2023-k]]))

In [35]:
df_credit_2016

,기업명,Point,공시일자,평가등급
0,엠에스오토텍,0.0,2016,14.0
1,대한전선,0.0,2016,15.0
2,우룡,0.0,2016,13.0
3,우리종합금융,0.0,2016,19.0
4,중국농업은행서울지점,0.0,2016,26.0
5,한국철강,0.0,2016,20.0
6,에스케이건설,0.0,2016,19.0
7,오뚜기,0.0,2016,23.0
8,한국투자저축은행,0.0,2016,20.0
9,엘아이지,0.0,2016,21.0


In [36]:
df_columns = ["기업코드","영업활동현금흐름","EBITDA","CAPEX","당기순이익","총차입부채","자산총계","매출액","EBIT",
       "차입부채_EBITDA","부채비율","차입금의존도"]

In [37]:
data_frames_2011 = [df_2011_1,df_2011_2,df_2011_3,df_2011_4,df_2011_5,df_2011_6,df_2011_7,df_2011_8,df_2011_9,df_2011_10,df_2011_11,]
df_merged_2011 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2011).dropna()
df_merged_2011.columns = df_columns

data_frames_2012 = [df_2012_1,df_2012_2,df_2012_3,df_2012_4,df_2012_5,df_2012_6,df_2012_7,df_2012_8,df_2012_9,df_2012_10,df_2012_11,]
df_merged_2012 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2012).dropna()
df_merged_2012.columns = df_columns

data_frames_2013 = [df_2013_1,df_2013_2,df_2013_3,df_2013_4,df_2013_5,df_2013_6,df_2013_7,df_2013_8,df_2013_9,df_2013_10,df_2013_11,]
df_merged_2013 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2013).dropna()
df_merged_2013.columns = df_columns

data_frames_2014 = [df_2014_1,df_2014_2,df_2014_3,df_2014_4,df_2014_5,df_2014_6,df_2014_7,df_2014_8,df_2014_9,df_2014_10,df_2014_11,]
df_merged_2014 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2014).dropna()
df_merged_2014.columns = df_columns

data_frames_2015 = [df_2015_1,df_2015_2,df_2015_3,df_2015_4,df_2015_5,df_2015_6,df_2015_7,df_2015_8,df_2015_9,df_2015_10,df_2015_11,]
df_merged_2015 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2015).dropna()
df_merged_2015.columns = df_columns

data_frames_2016 = [df_2016_1,df_2016_2,df_2016_3,df_2016_4,df_2016_5,df_2016_6,df_2016_7,df_2016_8,df_2016_9,df_2016_10,df_2016_11,]
df_merged_2016 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2016).dropna()
df_merged_2016.columns = df_columns

data_frames_2017 = [df_2017_1,df_2017_2,df_2017_3,df_2017_4,df_2017_5,df_2017_6,df_2017_7,df_2017_8,df_2017_9,df_2017_10,df_2017_11,]
df_merged_2017 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2017).dropna()
df_merged_2017.columns = df_columns

data_frames_2018 = [df_2018_1,df_2018_2,df_2018_3,df_2018_4,df_2018_5,df_2018_6,df_2018_7,df_2018_8,df_2018_9,df_2018_10,df_2018_11,]
df_merged_2018 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2018).dropna()
df_merged_2018.columns = df_columns

data_frames_2019 = [df_2019_1,df_2019_2,df_2019_3,df_2019_4,df_2019_5,df_2019_6,df_2019_7,df_2019_8,df_2019_9,df_2019_10,df_2019_11,]
df_merged_2019 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_2019).dropna()
df_merged_2019.columns = df_columns

In [38]:
df_merged_2011.head()

,기업코드,영업활동현금흐름,EBITDA,CAPEX,당기순이익,총차입부채,자산총계,매출액,EBIT,차입부채_EBITDA,부채비율,차입금의존도
0,000020,46244032.0,34559742.0,42917488,17679223.0,2.850000e+07,3.394315e+08,2.345620e+08,22102734.0,0.82,47.82,8.40
1,000040,-8429641.0,6582686.0,1033477,1904749.0,1.030472e+07,1.333378e+08,1.277202e+08,2493135.0,1.57,93.90,7.73
2,000050,-14647398.0,44173623.0,30859648,-1131177.0,4.125728e+08,1.352324e+09,3.481364e+08,22059512.0,9.34,114.76,30.51
3,000070,159327065.0,88142799.0,-216567063,168087785.0,3.435653e+08,1.884481e+09,1.180816e+09,43630799.0,3.90,45.22,18.23
4,000080,234454128.0,193650213.0,1604681503,70242546.0,1.347631e+09,3.573283e+09,1.373664e+09,124995582.0,6.96,151.06,37.71


In [39]:
df_merged_2011["FCF"] = df_merged_2011["영업활동현금흐름"] - df_merged_2011["CAPEX"]
df_merged_2011["EBIT_매출액"] = df_merged_2011["EBIT"]/df_merged_2011["매출액"]

df_merged_2012["FCF"] = df_merged_2012["영업활동현금흐름"] - df_merged_2012["CAPEX"]
df_merged_2012["EBIT_매출액"] = df_merged_2012["EBIT"]/df_merged_2012["매출액"]

df_merged_2013["FCF"] = df_merged_2013["영업활동현금흐름"] - df_merged_2013["CAPEX"]
df_merged_2013["EBIT_매출액"] = df_merged_2013["EBIT"]/df_merged_2013["매출액"]

df_merged_2014["FCF"] = df_merged_2014["영업활동현금흐름"] - df_merged_2014["CAPEX"]
df_merged_2014["EBIT_매출액"] = df_merged_2011["EBIT"]/df_merged_2011["매출액"]

df_merged_2015["FCF"] = df_merged_2015["영업활동현금흐름"] - df_merged_2015["CAPEX"]
df_merged_2015["EBIT_매출액"] = df_merged_2015["EBIT"]/df_merged_2015["매출액"]

df_merged_2016["FCF"] = df_merged_2016["영업활동현금흐름"] - df_merged_2016["CAPEX"]
df_merged_2016["EBIT_매출액"] = df_merged_2016["EBIT"]/df_merged_2016["매출액"]

df_merged_2017["FCF"] = df_merged_2017["영업활동현금흐름"] - df_merged_2017["CAPEX"]
df_merged_2017["EBIT_매출액"] = df_merged_2017["EBIT"]/df_merged_2017["매출액"]

df_merged_2018["FCF"] = df_merged_2018["영업활동현금흐름"] - df_merged_2018["CAPEX"]
df_merged_2018["EBIT_매출액"] = df_merged_2018["EBIT"]/df_merged_2018["매출액"]

df_merged_2019["FCF"] = df_merged_2019["영업활동현금흐름"] - df_merged_2019["CAPEX"]
df_merged_2019["EBIT_매출액"] = df_merged_2019["EBIT"]/df_merged_2019["매출액"]

In [40]:
data_frames_a_2011 = [df_merged_2011,df_market_2011,df_esg_2011]
df_merged_b_2011 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2011).dropna()

data_frames_a_2012 = [df_merged_2012,df_market_2012,df_esg_2012]
df_merged_b_2012 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2012).dropna()

data_frames_a_2013 = [df_merged_2013,df_market_2013,df_esg_2013]
df_merged_b_2013 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2013).dropna()

data_frames_a_2014 = [df_merged_2014,df_market_2014,df_esg_2014]
df_merged_b_2014 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2014).dropna()

data_frames_a_2015 = [df_merged_2015,df_market_2015,df_esg_2015]
df_merged_b_2015 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2015).dropna()

data_frames_a_2016 = [df_merged_2016,df_market_2016,df_esg_2016]
df_merged_b_2016 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2016).dropna()

data_frames_a_2017 = [df_merged_2017,df_market_2017,df_esg_2017]
df_merged_b_2017 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2017).dropna()

data_frames_a_2018 = [df_merged_2018,df_market_2018,df_esg_2018]
df_merged_b_2018 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2018).dropna()

data_frames_a_2019 = [df_merged_2019,df_market_2019,df_esg_2019]
df_merged_b_2019 = reduce(lambda left,right: pd.merge(left,right,on=['기업코드'], how='inner'), data_frames_a_2019).dropna()

In [41]:
df_merged_b_2011 = df_merged_b_2011.iloc[:,[0,7,4,12,5,6,13,10,11,16,18,19,20,21]]
df_merged_b_2012 = df_merged_b_2012.iloc[:,[0,7,4,12,5,6,13,10,11,16,18,19,20,21]]
df_merged_b_2013 = df_merged_b_2013.iloc[:,[0,7,4,12,5,6,13,10,11,16,18,19,20,21]]
df_merged_b_2014 = df_merged_b_2014.iloc[:,[0,7,4,12,5,6,13,10,11,16,18,19,20,21]]
df_merged_b_2015 = df_merged_b_2015.iloc[:,[0,7,4,12,5,6,13,10,11,16,18,19,20,21]]
df_merged_b_2016 = df_merged_b_2016.iloc[:,[0,7,4,12,5,6,13,10,11,16,18,19,20,21]]
df_merged_b_2017 = df_merged_b_2017.iloc[:,[0,7,4,12,5,6,13,10,11,16,18,19,20,21]]
df_merged_b_2018 = df_merged_b_2018.iloc[:,[0,7,4,12,5,6,13,10,11,16,18,19,20,21]]
df_merged_b_2019 = df_merged_b_2019.iloc[:,[0,7,4,12,5,6,13,10,11,16,18,19,20,21]]

In [42]:
df_merged_z_2012 = df_merged_b_2012[df_merged_b_2012["기업코드"].isin(list(set(df_merged_b_2011["기업코드"])&set(df_merged_b_2012["기업코드"])))].reset_index(drop=True)
df_merged_z_2011 = df_merged_b_2011[df_merged_b_2011["기업코드"].isin(list(set(df_merged_b_2011["기업코드"])&set(df_merged_b_2012["기업코드"])))].reset_index(drop=True).drop([217])

In [43]:
df_merged_z_2011[df_merged_z_2011['기업코드'] == "086280"] #7,5,25

,기업코드,매출액,당기순이익,FCF,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
218,086280,9.545995e+09,358806765.0,8.40114e+06,1.041864e+09,3.826580e+09,0.048446,156.52,27.23,7200000000000,40.0,30.0,40.0,30.0


In [44]:
df_market_2012[df_market_2012['기업코드'] == "004840"]

,기업코드,Name,Date,Marcap
388560,004840,동일벨트,20121018,205620000000
484050,004840,DRB동일,20121228,83013349500


In [45]:
df_merged_c_2012 = pd.DataFrame(columns=df_merged_b_2012.columns)
df_merged_c_2012["기업코드"] = df_merged_z_2012["기업코드"]
df_merged_c_2012['매출액'] = (df_merged_z_2012['매출액']) / (df_merged_z_2011['매출액'])
df_merged_c_2012['당기순이익'] = (df_merged_z_2012['당기순이익']) / (df_merged_z_2011['당기순이익'])
df_merged_c_2012["FCF"] = (df_merged_z_2012["FCF"])  / (df_merged_z_2011["FCF"])
df_merged_c_2012['총차입부채'] = (df_merged_z_2012['총차입부채'] ) / (df_merged_z_2011['총차입부채'])
df_merged_c_2012['자산총계'] = (df_merged_z_2012['자산총계'] ) / (df_merged_z_2011['자산총계'])
df_merged_c_2012['EBIT_매출액'] = (df_merged_z_2012['EBIT_매출액']) / (df_merged_z_2011['EBIT_매출액'])
df_merged_c_2012['부채비율'] = (df_merged_z_2012['부채비율'] ) / (df_merged_z_2011['부채비율'])
df_merged_c_2012['차입금의존도'] = (df_merged_z_2012['차입금의존도'] )/ (df_merged_z_2011['차입금의존도'])
df_merged_c_2012['Marcap'] = (df_merged_z_2012['Marcap'] ) / (df_merged_z_2011['Marcap'])
df_merged_c_2012['ESG등급'] = (df_merged_z_2012['ESG등급'] - df_merged_z_2011['ESG등급'])
df_merged_c_2012['지배구조'] = (df_merged_z_2012['지배구조'] - df_merged_z_2011['지배구조'])
df_merged_c_2012['사회'] = (df_merged_z_2012['사회'] - df_merged_z_2011['사회'])
df_merged_c_2012['환경'] = (df_merged_z_2012['환경'] - df_merged_z_2011['환경'])

In [46]:
df_merged_c_2012.describe()

,매출액,당기순이익,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
count,231.000000,231.000000,229.000000,231.000000,231.000000,231.000000,229.000000,231.000000,231.000000,231.000000,231.000000,231.000000
mean,8.797912,-18.198313,inf,7.449262,1.142734,1.410513,inf,13.088135,0.346320,0.000000,1.298701,2.077922
std,35.317556,401.199724,NaN,27.052858,2.217416,2.060849,NaN,71.759978,8.437054,10.591219,9.280740,8.022414
min,0.000911,-6048.907765,0.000000,0.019985,-15.031911,0.087037,0.000000,0.010295,-30.000000,-30.000000,-30.000000,-20.000000
25%,0.666907,0.023198,0.529302,0.677163,0.425106,0.778862,0.788232,0.592786,0.000000,0.000000,0.000000,0.000000
50%,1.049525,0.562949,1.018629,1.020677,0.811669,0.986665,1.017051,0.969298,0.000000,0.000000,0.000000,0.000000
75%,1.320891,1.758082,1.584983,1.341680,1.348240,1.226700,1.325508,1.626884,0.000000,0.000000,0.000000,10.000000
max,411.843715,528.919977,inf,275.776446,15.718321,25.904873,inf,911.960907,30.000000,40.000000,30.000000,30.000000


In [47]:
df_total_2013 = pd.merge(df_rating_2013,df_merged_c_2012.dropna(), on="기업코드", how="inner")

In [48]:
len(df_total_2013)

17

In [49]:
df_merged_z_2013 = df_merged_b_2013[df_merged_b_2013["기업코드"].isin(list(set(df_merged_b_2013["기업코드"])&set(df_merged_b_2012["기업코드"])))].reset_index(drop=True).drop([21,73,81,270,334,386])
df_merged_x_2012 = df_merged_b_2012[df_merged_b_2012["기업코드"].isin(list(set(df_merged_b_2013["기업코드"])&set(df_merged_b_2012["기업코드"])))].reset_index(drop=True).drop([7,5,25,148,152,231,403,418])

In [50]:
print(len(df_merged_z_2013))
print(len(df_merged_x_2012))

486
486


In [51]:
df_merged_z_2013[df_merged_z_2013['기업코드'] == "035420"] #7,5,25

,기업코드,매출액,당기순이익,FCF,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
387,035420,2.259139e+09,1.895289e+09,2.57839e+08,351680137.0,2.697747e+09,0.224218,82.86,13.04,23864979596000,40.0,40.0,40.0,30.0


In [52]:
#df_merged_z_2013.groupby("기업코드").size()

In [53]:
df_merged_c_2013 = pd.DataFrame(columns=df_merged_b_2012.columns)
df_merged_c_2013["기업코드"] = df_merged_z_2013["기업코드"]
df_merged_c_2013['매출액'] = (df_merged_z_2013['매출액'] ) / (df_merged_x_2012['매출액'])
df_merged_c_2013['당기순이익'] = (df_merged_z_2013['당기순이익'] )/ (df_merged_x_2012['당기순이익'])
df_merged_c_2013["FCF"] = (df_merged_z_2013["FCF"] ) / (df_merged_x_2012["FCF"])
df_merged_c_2013['총차입부채'] = (df_merged_z_2013['총차입부채']) / (df_merged_x_2012['총차입부채'])
df_merged_c_2013['자산총계'] = (df_merged_z_2013['자산총계']) / (df_merged_x_2012['자산총계'])
df_merged_c_2013['EBIT_매출액'] = (df_merged_z_2013['EBIT_매출액']) / (df_merged_x_2012['EBIT_매출액'])
df_merged_c_2013['부채비율'] = (df_merged_z_2013['부채비율'] ) / (df_merged_x_2012['부채비율'])
df_merged_c_2013['차입금의존도'] = (df_merged_z_2013['차입금의존도']) / (df_merged_x_2012['차입금의존도'])
df_merged_c_2013['Marcap'] = (df_merged_z_2013['Marcap'] ) / (df_merged_x_2012['Marcap'])
df_merged_c_2013['ESG등급'] = (df_merged_z_2013['ESG등급'] - df_merged_x_2012['ESG등급'])
df_merged_c_2013['지배구조'] = (df_merged_z_2013['지배구조'] - df_merged_x_2012['지배구조'])
df_merged_c_2013['사회'] = (df_merged_z_2013['사회'] - df_merged_x_2012['사회'])
df_merged_c_2013['환경'] = (df_merged_z_2013['환경'] - df_merged_x_2012['환경'])

In [54]:
df_rating_2014

,기업명,공시일자,평가등급,기업코드
0,두산인프라코어,2014,19.0,042670
1,화승알앤에이,2014,17.0,013520
2,화신,2014,19.0,010690
3,현대글로비스,2014,23.0,086280
4,금호타이어,2014,17.0,073240
5,한라홀딩스,2014,22.0,060980
6,남성,2014,13.0,004270
7,하나금융지주,2014,26.0,086790
8,웅진에너지,2014,8.0,103130
9,대웅제약,2014,22.0,069620


In [55]:
df_total_2014 = pd.merge(df_rating_2014,df_merged_c_2013.dropna(), on="기업코드", how="inner")

In [56]:
df_total_2014

,기업명,공시일자,평가등급,기업코드,매출액,당기순이익,FCF,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
0,두산인프라코어,2014,19.0,042670,44.619946,-4.132082,-0.331276,907.887589,49.705162,0.391116,8.530916,18.275093,15.226373,20.0,20.0,30.0,20.0
1,화승알앤에이,2014,17.0,013520,1.138607,-3.064506,3.20197,1.458161,1.183154,1.317395,1.593581,1.232273,1.357466,0.0,-10.0,0.0,0.0
2,화신,2014,19.0,010690,5.724288,0.133021,-1.74516,20.005208,4.457958,0.903376,2.559579,4.484709,7.555374,0.0,-10.0,0.0,0.0
3,현대글로비스,2014,23.0,086280,4.709551,5.294030,0.339815,1.879640,2.562491,0.924316,0.853927,0.733369,37.376989,10.0,10.0,10.0,0.0
4,금호타이어,2014,17.0,073240,48.227186,296.975588,55.1397,79.415801,45.772560,2.750106,2.180043,1.734894,75.802550,0.0,10.0,0.0,10.0
5,한라홀딩스,2014,22.0,060980,0.183574,1.096513,-0.633977,1.764617,1.258355,0.649993,1.241265,1.401964,0.961304,0.0,0.0,0.0,0.0
6,남성,2014,13.0,004270,1.074266,0.106764,-1.99931,0.742643,0.903638,-53.007613,0.880791,0.821829,1.198872,0.0,-10.0,0.0,0.0
7,대웅제약,2014,22.0,069620,4.518964,24.673139,-7.15262,0.430691,4.026375,3.511756,0.092208,0.106906,13.787032,0.0,10.0,0.0,-10.0
8,일진전기,2014,19.0,103590,6.997447,0.293286,-5.36549,23.232943,5.325892,0.136037,7.143861,4.359807,1.682085,0.0,-10.0,0.0,0.0
9,페이퍼코리아,2014,14.0,001020,0.813001,-2.073637,-2.61081,6.717939,1.360690,0.290633,5.744669,4.938640,0.909867,0.0,-10.0,0.0,0.0


In [57]:
df_merged_z_2014 = df_merged_b_2014[df_merged_b_2014["기업코드"].isin(list(set(df_merged_b_2014["기업코드"])&set(df_merged_b_2013["기업코드"])))].drop([242,255,360,489,512,]).reset_index(drop=True)
df_merged_x_2013 = df_merged_b_2013[df_merged_b_2013["기업코드"].isin(list(set(df_merged_b_2014["기업코드"])&set(df_merged_b_2013["기업코드"])))].drop([24,79,87,178,230,300,307,372,432,]).reset_index(drop=True)

In [58]:
print(len(df_merged_z_2014))
print(len(df_merged_x_2013))

481
481


In [59]:
k = pd.DataFrame(df_merged_x_2013.groupby("기업코드").size())
k[k[0]==2]

,0
기업코드,


In [60]:
df_merged_x_2013[df_merged_x_2013['기업코드'] == "011420"] #24,79,87,178,230,300,307,372,432,/242,255,360,489,512,

,기업코드,매출액,당기순이익,FCF,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
262,011420,46437304.0,-1736750.0,-1.71914e+06,4243306.0,37636664.0,0.01609,75.11,11.27,36890157120,30.0,20.0,30.0,30.0


In [61]:
df_merged_c_2014 = pd.DataFrame(columns=df_merged_b_2012.columns)
df_merged_c_2014["기업코드"] = df_merged_z_2014["기업코드"]
df_merged_c_2014['매출액'] = (df_merged_z_2014['매출액'] ) / (df_merged_x_2013['매출액'])
df_merged_c_2014['당기순이익'] = (df_merged_z_2014['당기순이익'])/(df_merged_x_2013['당기순이익'])
df_merged_c_2014["FCF"] = (df_merged_z_2014["FCF"] ) /(df_merged_x_2013["FCF"])
df_merged_c_2014['총차입부채'] = (df_merged_z_2014['총차입부채'] ) / (df_merged_x_2013['총차입부채'])
df_merged_c_2014['자산총계'] = (df_merged_z_2014['자산총계'] ) / (df_merged_x_2013['자산총계'])
df_merged_c_2014['EBIT_매출액'] = (df_merged_z_2014['EBIT_매출액']) / (df_merged_x_2013['EBIT_매출액'])
df_merged_c_2014['부채비율'] = (df_merged_z_2014['부채비율'] ) / (df_merged_x_2013['부채비율'])
df_merged_c_2014['차입금의존도'] = (df_merged_z_2014['차입금의존도']) / (df_merged_x_2013['차입금의존도'])
df_merged_c_2014['Marcap'] = (df_merged_z_2014['Marcap']) / (df_merged_x_2013['Marcap'])
df_merged_c_2014['ESG등급'] = (df_merged_z_2014['ESG등급'] - df_merged_x_2013['ESG등급'])
df_merged_c_2014['지배구조'] = (df_merged_z_2014['지배구조'] - df_merged_x_2013['지배구조'])
df_merged_c_2014['사회'] = (df_merged_z_2014['사회'] - df_merged_x_2013['사회'])
df_merged_c_2014['환경'] = (df_merged_z_2014['환경'] - df_merged_x_2013['환경'])

In [62]:
df_total_2015 = pd.merge(df_rating_2015,df_merged_c_2014.dropna(), on="기업코드", how="inner")

In [63]:
df_total_2015.describe()

,Point,공시일자,평가등급,매출액,당기순이익,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
count,81.000000,81.0,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000,81.000000
mean,0.185185,2015.0,20.320988,1.021231,1.500016,1.095278,1.046229,0.901709,1.174869,1.039076,1.164135,0.123457,0.987654,0.000000,-0.246914
std,0.390868,0.0,3.089608,0.087312,4.284712,0.389144,0.124613,5.988620,1.701740,0.302361,0.683898,4.608098,7.177210,5.244044,5.695461
min,0.000000,2015.0,8.000000,0.842220,-4.974584,0.033534,0.330784,-32.536174,0.289192,0.034173,0.333693,-10.000000,-20.000000,-10.000000,-20.000000
25%,0.000000,2015.0,19.000000,0.976182,0.163943,0.943202,0.999787,0.737189,0.905706,0.919037,0.797194,0.000000,0.000000,0.000000,0.000000
50%,0.000000,2015.0,20.000000,1.017672,0.784267,1.035382,1.043676,1.070446,0.981807,0.986958,0.982979,0.000000,0.000000,0.000000,0.000000
75%,0.000000,2015.0,22.000000,1.062130,1.218072,1.154306,1.098933,1.438928,1.059024,1.095982,1.287742,0.000000,0.000000,0.000000,0.000000
max,1.000000,2015.0,26.000000,1.266425,27.173609,3.367555,1.331980,34.812689,16.210018,2.809459,5.650776,10.000000,30.000000,20.000000,10.000000


In [64]:
df_merged_z_2015 = df_merged_b_2015[df_merged_b_2015["기업코드"].isin(list(set(df_merged_b_2014["기업코드"])&set(df_merged_b_2015["기업코드"])))].reset_index(drop=True).drop([55,260,272,281,323,377,379,467])
df_merged_x_2014 = df_merged_b_2014[df_merged_b_2014["기업코드"].isin(list(set(df_merged_b_2014["기업코드"])&set(df_merged_b_2015["기업코드"])))].reset_index(drop=True).drop([211,222,314,432,454])

In [65]:
print(len(df_merged_z_2015))
print(len(df_merged_x_2014))

486
486


In [66]:
k = pd.DataFrame(df_merged_z_2015.groupby("기업코드").size())
k[k[0]==2]

,0
기업코드,


In [67]:
df_merged_z_2015[df_merged_z_2015['기업코드'] == "102260"] #211,222,314,432,454 / 55,260,272,281,323,377,379,467

,기업코드,매출액,당기순이익,FCF,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
468,102260,871224899.0,43556795.0,2.61585e+07,242378484.0,801695229.0,0.076545,101.18,30.23,290393358300,30.0,20.0,30.0,30.0


In [68]:
df_merged_c_2015 = pd.DataFrame(columns=df_merged_b_2012.columns)
df_merged_c_2015["기업코드"] = df_merged_z_2015["기업코드"]
df_merged_c_2015['매출액'] = (df_merged_z_2015['매출액'] ) / (df_merged_x_2014['매출액'])
df_merged_c_2015['당기순이익'] = (df_merged_z_2015['당기순이익'] )/ (df_merged_x_2014['당기순이익'])
df_merged_c_2015["FCF"] = (df_merged_z_2015["FCF"] ) / (df_merged_x_2014["FCF"])
df_merged_c_2015['총차입부채'] = (df_merged_z_2015['총차입부채'] ) / (df_merged_x_2014['총차입부채'])
df_merged_c_2015['자산총계'] = (df_merged_z_2015['자산총계'] ) / (df_merged_x_2014['자산총계'])
df_merged_c_2015['EBIT_매출액'] = (df_merged_z_2015['EBIT_매출액'] ) / (df_merged_x_2014['EBIT_매출액'])
df_merged_c_2015['부채비율'] = (df_merged_z_2015['부채비율'] ) / (df_merged_x_2014['부채비율'])
df_merged_c_2015['차입금의존도'] = (df_merged_z_2015['차입금의존도'] ) / (df_merged_x_2014['차입금의존도'])
df_merged_c_2015['Marcap'] = (df_merged_z_2015['Marcap'] ) / (df_merged_x_2014['Marcap'])
df_merged_c_2015['ESG등급'] = (df_merged_z_2015['ESG등급'] - df_merged_x_2014['ESG등급'])
df_merged_c_2015['지배구조'] = (df_merged_z_2015['지배구조'] - df_merged_x_2014['지배구조'])
df_merged_c_2015['사회'] = (df_merged_z_2015['사회'] - df_merged_x_2014['사회'])
df_merged_c_2015['환경'] = (df_merged_z_2015['환경'] - df_merged_x_2014['환경'])

In [69]:
df_total_2016 = pd.merge(df_rating_2016,df_merged_c_2015.dropna(), on="기업코드", how="inner")

In [70]:
df_total_2016.describe()

,Point,공시일자,평가등급,매출액,당기순이익,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
count,76.000000,76.0,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000
mean,0.131579,2016.0,20.447368,17.205953,28.028748,inf,28.964490,2.238109,74.159791,inf,19.060292,0.789474,-1.842105,2.368421,3.684211
std,0.340279,0.0,2.881642,37.909583,249.555902,NaN,75.894576,7.050609,623.107785,NaN,44.609178,6.274426,10.672148,7.278905,8.301342
min,0.000000,2016.0,8.000000,0.184397,-619.263273,0.000000,0.158798,-7.286142,0.137999,0.000000,0.139103,-10.000000,-30.000000,-10.000000,-20.000000
25%,0.000000,2016.0,19.000000,1.043552,-0.238138,1.038544,1.106718,0.418442,0.957584,0.914970,0.874673,0.000000,-10.000000,0.000000,0.000000
50%,0.000000,2016.0,20.500000,2.879009,1.382186,3.596478,3.190440,0.932096,1.300327,1.119816,2.100216,0.000000,0.000000,0.000000,0.000000
75%,0.000000,2016.0,22.000000,8.726386,8.328560,75.748216,16.960149,1.566297,3.322539,2.503872,9.991937,0.000000,0.000000,10.000000,10.000000
max,1.000000,2016.0,26.000000,209.848745,1850.374181,inf,426.185761,49.153783,5434.742122,inf,251.898134,20.000000,40.000000,30.000000,30.000000


In [71]:
df_merged_z_2016 = df_merged_b_2016[df_merged_b_2016["기업코드"].isin(list(set(df_merged_b_2015["기업코드"])&set(df_merged_b_2016["기업코드"])))].reset_index(drop=True).drop([14,32,137,189,234,435,486 ])
df_merged_x_2015 = df_merged_b_2015[df_merged_b_2015["기업코드"].isin(list(set(df_merged_b_2015["기업코드"])&set(df_merged_b_2016["기업코드"])))].reset_index(drop=True).drop([62,296,303,313,316,358,393,419,421,512])

In [72]:
print(len(df_merged_z_2016))
print(len(df_merged_x_2015))

538
538


In [73]:
k = pd.DataFrame(df_merged_x_2015.groupby("기업코드").size())
k[k[0]==2]

,0
기업코드,


In [74]:
df_merged_x_2015[df_merged_x_2015['기업코드'] == "102260"] #14,32,137,189,234,435,486 / 62,296,303,313,316,358,393,419,421,512

,기업코드,매출액,당기순이익,FCF,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
513,102260,871224899.0,43556795.0,2.61585e+07,242378484.0,801695229.0,0.076545,101.18,30.23,290393358300,30.0,20.0,30.0,30.0


In [75]:
df_merged_c_2016 = pd.DataFrame(columns=df_merged_b_2012.columns)
df_merged_c_2016["기업코드"] = df_merged_z_2016["기업코드"]
df_merged_c_2016['매출액'] = (df_merged_z_2016['매출액'] ) / (df_merged_x_2015['매출액'])
df_merged_c_2016['당기순이익'] = (df_merged_z_2016['당기순이익'] )/ (df_merged_x_2015['당기순이익'])
df_merged_c_2016["FCF"] = (df_merged_z_2016["FCF"] ) / (df_merged_x_2015["FCF"])
df_merged_c_2016['총차입부채'] = (df_merged_z_2016['총차입부채']) / (df_merged_x_2015['총차입부채'])
df_merged_c_2016['자산총계'] = (df_merged_z_2016['자산총계']) / (df_merged_x_2015['자산총계'])
df_merged_c_2016['EBIT_매출액'] = (df_merged_z_2016['EBIT_매출액'] ) / (df_merged_x_2015['EBIT_매출액'])
df_merged_c_2016['부채비율'] = (df_merged_z_2016['부채비율'] ) / (df_merged_x_2015['부채비율'])
df_merged_c_2016['차입금의존도'] = (df_merged_z_2016['차입금의존도'] ) / (df_merged_x_2015['차입금의존도'])
df_merged_c_2016['Marcap'] = (df_merged_z_2016['Marcap'] ) / (df_merged_x_2015['Marcap'])
df_merged_c_2016['ESG등급'] = (df_merged_z_2016['ESG등급'] - df_merged_x_2015['ESG등급'])
df_merged_c_2016['지배구조'] = (df_merged_z_2016['지배구조'] - df_merged_x_2015['지배구조'])
df_merged_c_2016['사회'] = (df_merged_z_2016['사회'] - df_merged_x_2015['사회'])
df_merged_c_2016['환경'] = (df_merged_z_2016['환경'] - df_merged_x_2015['환경'])

In [76]:
df_total_2017 = pd.merge(df_rating_2017,df_merged_c_2016.dropna(), on="기업코드", how="inner")

In [77]:
df_total_2017

,기업명,Point,공시일자,평가등급,기업코드,매출액,당기순이익,FCF,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
0,한샘,0.0,2017,21.0,009240,39.855012,113.056334,-300.278,0.875691,14.082417,1.646171,0.549971,0.062063,82.439434,0.0,20.0,0.0,0.0
1,한라,0.0,2017,17.0,014790,0.987363,-0.088849,-0.590198,0.646428,0.907431,3.124951,0.806488,0.712354,1.088290,10.0,20.0,10.0,0.0
2,금호타이어,1.0,2017,17.0,073240,1.472182,-0.229591,-1.05748,1.278434,1.059090,0.361747,1.911166,1.207255,2.008862,0.0,-30.0,20.0,0.0
3,도화엔지니어링,0.0,2017,20.0,002150,1.510133,0.120006,-0.320301,0.007856,0.983458,0.818238,1.011650,0.008073,1.133280,0.0,10.0,0.0,0.0
4,신원,0.0,2017,17.0,009270,0.339820,-0.349270,-2.11136,0.197750,0.143567,0.777311,1.004775,1.377539,0.255132,-10.0,-30.0,0.0,-10.0
5,휴켐스,0.0,2017,21.0,069260,3.198399,1.550961,4.45412,1.530410,1.418574,0.677979,0.909905,1.078826,3.745959,10.0,0.0,10.0,20.0
6,한화케미칼,0.0,2017,21.0,009830,8.175709,53.994266,-129.469,26.760394,19.269671,8.095752,0.952229,1.388558,46.580446,10.0,10.0,30.0,10.0
7,하이트진로홀딩스,0.0,2017,19.0,000140,0.983557,1.034965,1.10336,0.900079,0.975011,0.926560,0.863011,0.923045,0.915441,0.0,10.0,0.0,0.0
8,한국전력공사,0.0,2017,26.0,015760,1.020908,0.532806,1.01152,0.911930,1.014719,1.036053,0.908135,0.898820,0.881000,0.0,-20.0,20.0,0.0
9,롯데쇼핑,0.0,2017,24.0,023530,315.227455,71.356530,221.625,2028.472387,613.985553,0.601038,5.236157,3.304099,132.758725,10.0,0.0,20.0,20.0


In [78]:
df_total_2017.describe()

,Point,공시일자,평가등급,매출액,당기순이익,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
count,88.000000,88.0,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000,88.000000
mean,0.159091,2017.0,20.329545,96.813072,-65.568030,inf,45.557447,1.498309,2.934025,inf,23.793011,4.090909,2.159091,7.159091,5.795455
std,0.367857,0.0,3.087690,602.960875,679.963499,NaN,133.405806,2.732128,4.828225,NaN,61.511162,7.825153,13.082309,11.739399,9.434784
min,0.000000,2017.0,8.000000,0.039599,-6278.588651,0.000000,0.048054,-6.698760,0.025275,0.000000,0.025236,-20.000000,-30.000000,-30.000000,-20.000000
25%,0.000000,2017.0,19.000000,1.032038,-0.118836,0.908967,1.024744,0.673930,0.857480,0.710266,0.899887,0.000000,0.000000,0.000000,0.000000
50%,0.000000,2017.0,21.000000,4.935885,0.839685,5.774863,4.580538,0.960566,1.105706,1.012378,3.488243,0.000000,0.000000,10.000000,0.000000
75%,0.000000,2017.0,22.000000,26.059941,9.534775,49.025429,22.237018,1.651175,2.702917,1.717388,16.229937,10.000000,10.000000,10.000000,10.000000
max,1.000000,2017.0,26.000000,5646.916967,647.639296,inf,944.582640,13.432627,29.367201,inf,358.590478,30.000000,40.000000,30.000000,20.000000


In [79]:
df_merged_z_2017 = df_merged_b_2017[df_merged_b_2017["기업코드"].isin(list(set(df_merged_b_2016["기업코드"])&set(df_merged_b_2017["기업코드"])))].reset_index(drop=True).drop([36, 106, 301, 303, 308, 391, 439, 449,477])
df_merged_x_2016 = df_merged_b_2016[df_merged_b_2016["기업코드"].isin(list(set(df_merged_b_2016["기업코드"])&set(df_merged_b_2017["기업코드"])))].reset_index(drop=True).drop([14, 32, 48, 136, 190, 238, 449, 503, 570,392,393])

In [80]:
print(len(df_merged_z_2017))
print(len(df_merged_x_2016))

563
563


In [81]:
k = pd.DataFrame(df_merged_z_2017.groupby("기업코드").size())
k[k[0]==2]

,0
기업코드,


In [82]:
df_merged_z_2017[df_merged_z_2017['기업코드'] == "069640"] #14, 32, 48, 136, 190, 238, 449, 503, 570,392,393 / 36, 106, 301, 303, 308, 391, 439, 449,477

,기업코드,매출액,당기순이익,FCF,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
478,069640,328898003.0,7471384.0,-694616,13004376.0,212873830.0,0.028984,34.24,6.11,153620931800,30.0,30.0,30.0,30.0


In [83]:
df_merged_c_2017 = pd.DataFrame(columns=df_merged_b_2012.columns)
df_merged_c_2017["기업코드"] = df_merged_z_2017["기업코드"]
df_merged_c_2017['매출액'] = (df_merged_z_2017['매출액'] ) / (df_merged_x_2016['매출액'])
df_merged_c_2017['당기순이익'] = (df_merged_z_2017['당기순이익'] )/ (df_merged_x_2016['당기순이익'])
df_merged_c_2017["FCF"] = (df_merged_z_2017["FCF"] ) / (df_merged_x_2016["FCF"])
df_merged_c_2017['총차입부채'] = (df_merged_z_2017['총차입부채'] ) /(df_merged_x_2016['총차입부채'])
df_merged_c_2017['자산총계'] = (df_merged_z_2017['자산총계'] ) / (df_merged_x_2016['자산총계'])
df_merged_c_2017['EBIT_매출액'] = (df_merged_z_2017['EBIT_매출액'] ) / (df_merged_x_2016['EBIT_매출액'])
df_merged_c_2017['부채비율'] = (df_merged_z_2017['부채비율'] ) / (df_merged_x_2016['부채비율'])
df_merged_c_2017['차입금의존도'] = (df_merged_z_2017['차입금의존도'] ) / (df_merged_x_2016['차입금의존도'])
df_merged_c_2017['Marcap'] = (df_merged_z_2017['Marcap'] ) / (df_merged_x_2016['Marcap'])
df_merged_c_2017['ESG등급'] = (df_merged_z_2017['ESG등급'] - df_merged_x_2016['ESG등급'])
df_merged_c_2017['지배구조'] = (df_merged_z_2017['지배구조'] - df_merged_x_2016['지배구조'])
df_merged_c_2017['사회'] = (df_merged_z_2017['사회'] - df_merged_x_2016['사회'])
df_merged_c_2017['환경'] = (df_merged_z_2017['환경'] - df_merged_x_2016['환경'])

In [84]:
df_total_2018 = pd.merge(df_rating_2018,df_merged_c_2017, on="기업코드", how="inner")

In [85]:
df_total_2018

,기업명,Point,공시일자,평가등급,기업코드,매출액,당기순이익,FCF,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
0,한일시멘트,0.0,2018,21,003300,1.431656,-3.822220,3.53429,4.000268,3.480647,13.339206,0.260070,1.149404,7.898315,0.0,10.0,0.0,10.0
1,롯데푸드,0.0,2018,23,002270,16.344966,20.171705,-71.9414,3.144366,4.073727,0.665181,0.896430,0.771756,12.273022,0.0,0.0,30.0,20.0
2,하이트진로,0.0,2018,20,000080,0.999829,0.331258,1.26398,0.961063,1.031317,0.703714,1.152150,0.931669,1.142180,10.0,10.0,10.0,0.0
3,현대중공업,0.0,2018,19,009540,39.524690,399.709566,1080.4,58.474884,65.713196,0.014322,1.193630,0.889848,33.665664,10.0,10.0,-20.0,20.0
4,한솔테크닉스,1.0,2018,18,004710,0.305633,0.931485,0.103436,0.137700,0.136447,1.620012,0.796533,1.009245,0.897105,0.0,0.0,-10.0,0.0
5,롯데칠성음료,1.0,2018,23,005300,12.892253,7.967418,8.51656,34.847766,17.678149,0.285591,2.137731,1.971617,10.348915,10.0,0.0,0.0,20.0
6,만도,0.0,2018,22,204320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,풍산,0.0,2018,20,103140,24.169309,14.081828,3.47357,462.167864,13.026421,0.709146,6.382353,35.642105,3.546054,10.0,10.0,0.0,10.0
8,팬오션,0.0,2018,19,028670,7.229033,12.426100,40.9122,26.533154,13.030298,2.153022,0.616038,2.036340,25.516434,0.0,10.0,0.0,0.0
9,포스코대우,0.0,2018,22,047050,84.561784,6.809483,7.13277,17.575692,13.672106,0.235428,2.369170,1.285589,6.234249,10.0,10.0,20.0,0.0


In [86]:
df_merged_z_2018 = df_merged_b_2018[df_merged_b_2018["기업코드"].isin(list(set(df_merged_b_2017["기업코드"])&set(df_merged_b_2018["기업코드"])))].reset_index(drop=True).drop([212,315,319,346,452,474,509,518,230,231])
df_merged_x_2017 = df_merged_b_2017[df_merged_b_2017["기업코드"].isin(list(set(df_merged_b_2017["기업코드"])&set(df_merged_b_2018["기업코드"])))].reset_index(drop=True).drop([36,111,307,309,314,401,449,462,488,594,596])

In [87]:
print(len(df_merged_z_2018))
print(len(df_merged_x_2017))

593
593


In [88]:
k = pd.DataFrame(df_merged_x_2017.groupby("기업코드").size())
k[k[0]==2]

,0
기업코드,


In [89]:
df_merged_x_2017[df_merged_x_2017['기업코드'] == "227840"]# 212,315,319,346,452,474,509,518,230,231 / 36,111,307,309,314,401,449,462,488,594,596

,기업코드,매출액,당기순이익,FCF,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
597,227840,145238609.0,12026649.0,7.55999e+06,0.0,198540566.0,0.060146,11.31,0.0,132872205600,30.0,30.0,30.0,30.0


In [90]:
df_merged_c_2018 = pd.DataFrame(columns=df_merged_b_2012.columns)
df_merged_c_2018["기업코드"] = df_merged_z_2018["기업코드"]
df_merged_c_2018['매출액'] = (df_merged_z_2018['매출액'] ) / (df_merged_x_2017['매출액'])
df_merged_c_2018['당기순이익'] = (df_merged_z_2018['당기순이익']  )/ (df_merged_x_2017['당기순이익'])
df_merged_c_2018["FCF"] = (df_merged_z_2018["FCF"] ) / (df_merged_x_2017["FCF"])
df_merged_c_2018['총차입부채'] = (df_merged_z_2018['총차입부채'] ) / (df_merged_x_2017['총차입부채'])
df_merged_c_2018['자산총계'] = (df_merged_z_2018['자산총계']) / (df_merged_x_2017['자산총계'])
df_merged_c_2018['EBIT_매출액'] = (df_merged_z_2018['EBIT_매출액'] ) / (df_merged_x_2017['EBIT_매출액'])
df_merged_c_2018['부채비율'] = (df_merged_z_2018['부채비율'] ) / (df_merged_x_2017['부채비율'])
df_merged_c_2018['차입금의존도'] = (df_merged_z_2018['차입금의존도']) / (df_merged_x_2017['차입금의존도'])
df_merged_c_2018['Marcap'] = (df_merged_z_2018['Marcap'] ) / (df_merged_x_2017['Marcap'])
df_merged_c_2018['ESG등급'] = (df_merged_z_2018['ESG등급'] - df_merged_x_2017['ESG등급'])
df_merged_c_2018['지배구조'] = (df_merged_z_2018['지배구조'] - df_merged_x_2017['지배구조'])
df_merged_c_2018['사회'] = (df_merged_z_2018['사회'] - df_merged_x_2017['사회'])
df_merged_c_2018['환경'] = (df_merged_z_2018['환경'] - df_merged_x_2017['환경'])

In [91]:
df_total_2019 = pd.merge(df_rating_2019,df_merged_c_2018, on="기업코드", how="inner")

In [92]:
df_total_2019

,기업명,Point,공시일자,평가등급,기업코드,매출액,당기순이익,FCF,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
0,대림산업,1,2019,22,000210,0.890475,1.334824,4.89944,8.430406e-01,0.957565,1.739029,0.828249,0.880204,1.243932,0.0,0.0,0.0,0.0
1,한솔테크닉스,0,2019,18,004710,0.255501,-3.884272,0.0538476,1.657888e-01,0.138492,0.996101,0.889960,1.197028,0.300127,0.0,0.0,0.0,0.0
2,해태제과식품,0,2019,20,101530,0.876767,-0.411544,-6.75959,1.128700e+00,0.930184,7.180128,1.687835,1.213545,1.062131,0.0,0.0,10.0,0.0
3,대한항공,0,2019,18,003490,8.576432,-0.534778,0.502977,1.788774e+01,7.381037,0.311874,9.791507,2.423398,1.667884,-10.0,-10.0,0.0,-10.0
4,한미약품,0,2019,21,128940,2.023970,5.776691,89.6427,4.030507e+00,3.321425,2.770296,1.042035,1.213264,58.805033,10.0,-10.0,20.0,10.0
5,풍산,0,2019,20,103140,3.641150,-29.741483,-0.202612,3.908691e+00,3.525644,4.948661,0.677101,1.108696,4.655480,10.0,0.0,10.0,10.0
6,한솔제지,0,2019,20,213500,1.735866,0.539490,0.340912,8.524492e+04,1.102204,0.718960,1.494754,inf,0.269870,20.0,20.0,20.0,20.0
7,삼천리,0,2019,24,004690,3.110671,0.399146,-0.414263,9.792025e+00,3.229931,0.386368,5.942100,3.032161,0.737277,10.0,20.0,10.0,0.0
8,한국콜마,0,2019,19,161890,5.226881,33.527093,159.955,3.228680e+01,8.050406,2.559370,3.172639,4.011886,14.998726,0.0,-10.0,10.0,10.0
9,팬오션,0,2019,19,028670,1.142157,1.052168,-0.0700476,9.221863e-01,1.057810,0.915550,0.890115,0.871893,0.843503,10.0,0.0,0.0,0.0


In [93]:
df_merged_z_2019 = df_merged_b_2019[df_merged_b_2019["기업코드"].isin(list(set(df_merged_b_2018["기업코드"])&set(df_merged_b_2019["기업코드"])))].reset_index(drop=True).drop([11,50,55,91,370,446,462,559,565])
df_merged_x_2018 = df_merged_b_2018[df_merged_b_2018["기업코드"].isin(list(set(df_merged_b_2018["기업코드"])&set(df_merged_b_2019["기업코드"])))].reset_index(drop=True).drop([205,305,309,337,368,440,462,497,505,553,222,223])

In [94]:
print(len(df_merged_z_2019))
print(len(df_merged_x_2018))

580
580


In [95]:
k = pd.DataFrame(df_merged_z_2019.groupby("기업코드").size())
k[k[0]==2]

,0
기업코드,


In [96]:
df_merged_z_2019[df_merged_z_2019['기업코드'] == "181710"] #205,305,309,337,368,440,462,497,505,553,222,223 / 11,50,55,91,370,446,462,559,565

,기업코드,매출액,당기순이익,FCF,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
566,181710,1.488597e+09,23173213.0,129309272.0,140196249.0,2.500321e+09,0.058227,36.73,5.61,1306943670000,30.0,40.0,30.0,20.0


In [97]:
df_merged_c_2019 = pd.DataFrame(columns=df_merged_b_2012.columns)

In [98]:
df_merged_c_2019.columns

Index(['기업코드', '매출액', '당기순이익', 'FCF', '총차입부채',
       '자산총계', 'EBIT_매출액', '부채비율', '차입금의존도', 'Marcap',
       'ESG등급', '지배구조', '사회', '환경'],
      dtype='object')

In [99]:
df_merged_c_2019 = pd.DataFrame(columns=df_merged_b_2012.columns)
df_merged_c_2019["기업코드"] = df_merged_z_2019["기업코드"]
df_merged_c_2019['매출액'] = (df_merged_z_2019['매출액'] - df_merged_x_2018['매출액']) / np.abs(df_merged_x_2018['매출액'])
df_merged_c_2019['당기순이익'] = (df_merged_z_2019['당기순이익'] - df_merged_x_2018['당기순이익'] )/ np.abs(df_merged_x_2018['당기순이익'])
df_merged_c_2019["FCF"] = (df_merged_z_2019["FCF"] - df_merged_x_2018["FCF"]) / np.abs(df_merged_x_2018["FCF"])
df_merged_c_2019['총차입부채'] = (df_merged_z_2019['총차입부채']) / (df_merged_x_2018['총차입부채'])
df_merged_c_2019['자산총계'] = (df_merged_z_2019['자산총계'] - df_merged_x_2018['자산총계']) / np.abs(df_merged_x_2018['자산총계'])
df_merged_c_2019['EBIT_매출액'] = (df_merged_z_2019['EBIT_매출액'] - df_merged_x_2018['EBIT_매출액']) / np.abs(df_merged_x_2018['EBIT_매출액'])
df_merged_c_2019['부채비율'] = (df_merged_z_2019['부채비율'] - df_merged_x_2018['부채비율']) / np.abs(df_merged_x_2018['부채비율'])
df_merged_c_2019['차입금의존도'] = (df_merged_z_2019['차입금의존도'] - df_merged_x_2018['차입금의존도']) / np.abs(df_merged_x_2018['차입금의존도'])
df_merged_c_2019['Marcap'] = (df_merged_z_2019['Marcap'] - df_merged_x_2018['Marcap']) / np.abs(df_merged_x_2018['Marcap'])
df_merged_c_2019['ESG등급'] = (df_merged_z_2019['ESG등급'] - df_merged_x_2018['ESG등급'])
df_merged_c_2019['지배구조'] = (df_merged_z_2019['지배구조'] - df_merged_x_2018['지배구조'])
df_merged_c_2019['사회'] = (df_merged_z_2019['사회'] - df_merged_x_2018['사회'])
df_merged_c_2019['환경'] = (df_merged_z_2019['환경'] - df_merged_x_2018['환경'])

In [100]:
df_rating_2020 = merge_code(df_credit_2020, df_esg_2019)

In [101]:
df_total_2020 = pd.merge(df_rating_2020,df_merged_c_2019, on="기업코드", how="inner")

In [102]:
df_total_2020

,기업명,Point,공시일자,평가등급,기업코드,매출액,당기순이익,FCF,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
0,송원산업,0,2020,19,004430,0.142642,0.800734,5.86758,2.243231e+00,0.476710,0.480795,0.799419,0.519053,1.780895,-10.0,0.0,0.0,-10.0
1,녹십자,0,2020,22,006280,3.033143,0.615475,-13.666,4.952651e+00,2.569211,-0.512173,-0.689980,0.387732,1.192138,10.0,10.0,10.0,-10.0
2,한화,0,2020,21,000880,613.752427,9.289438,82.0751,1.546653e+02,908.351889,1.166713,3.754407,-0.829932,9.267289,10.0,20.0,10.0,0.0
3,풍산,0,2020,20,103140,9.584117,0.177671,2.68551,inf,1.911287,1.205393,5.254038,inf,1.337089,10.0,10.0,0.0,10.0
4,호텔신라,0,2020,23,008770,28.022510,15.663905,33.9817,3.139734e+01,12.947766,-0.197659,3.585125,1.250635,53.911085,20.0,30.0,20.0,20.0
5,한화에어로스페이스,0,2020,22,012450,0.182103,2.297946,0.97694,1.254401e+00,0.177339,1.628429,0.134861,0.065700,0.068514,0.0,20.0,10.0,-10.0
6,케이티스카이라이프,0,2020,22,053210,-0.975353,-0.963136,1.04088,1.640543e-03,-0.970693,0.252848,-0.698167,-0.943961,-0.982976,-10.0,0.0,-20.0,-30.0
7,롯데칠성음료,0,2020,23,005300,2.582879,-15.292005,6.01631,7.176457e+00,5.605746,3.809227,-0.187976,0.086561,4.838599,10.0,0.0,20.0,0.0
8,롯데푸드,0,2020,23,002270,-0.083424,-0.177865,0.767187,2.099398e-01,-0.503589,0.282567,-0.419478,-0.577194,-0.274393,10.0,10.0,10.0,10.0
9,영원무역,0,2020,22,111770,0.394457,4.499488,13.0951,9.201324e-01,1.651805,3.410188,-0.581008,-0.653085,0.911420,-10.0,0.0,-10.0,-10.0


In [103]:
df_sum_2015_2020 = pd.concat([df_total_2015,df_total_2016,df_total_2017,df_total_2018,df_total_2019,df_total_2020]).replace(np.inf,np.nan).dropna().drop([30])

In [104]:
df=df_sum_2015_2020.reset_index(drop=True)[["Point",'매출액','당기순이익','FCF','총차입부채','자산총계','EBIT_매출액','부채비율',
                                           '차입금의존도','Marcap','ESG등급','지배구조','사회','환경']]

In [105]:
df.describe()

,Point,매출액,당기순이익,총차입부채,자산총계,EBIT_매출액,부채비율,차입금의존도,Marcap,ESG등급,지배구조,사회,환경
count,412.000000,412.000000,412.000000,412.000000,412.000000,412.000000,412.000000,412.000000,412.000000,412.000000,412.000000,412.000000,412.000000
mean,0.138350,36.367912,-0.165630,265.711575,25.959822,1.563439,2.053734,15.863234,13.388338,2.257282,2.014563,3.762136,2.548544
std,0.345686,303.782626,357.632014,2468.908648,107.719605,5.920806,3.096112,183.250781,38.571001,8.194819,11.612971,11.015217,9.413976
min,0.000000,-0.975353,-6278.588651,0.000000,-0.970693,-48.299784,-0.891551,-0.971369,-0.986551,-20.000000,-30.000000,-30.000000,-30.000000
25%,0.000000,0.990153,-0.150375,0.917682,0.994006,0.453206,0.816506,0.806707,0.854680,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.262478,0.936372,1.269980,1.221886,0.946862,1.019471,1.008720,1.434457,0.000000,0.000000,0.000000,0.000000
75%,0.000000,8.165931,5.525862,15.577758,8.093216,1.648308,1.878832,1.439559,8.237244,10.000000,10.000000,10.000000,10.000000
max,1.000000,5646.916967,2556.982083,47729.936848,1336.322247,54.229326,29.367201,3478.000000,358.590478,30.000000,40.000000,30.000000,30.000000


In [106]:
df.columns

Index(['Point', '매출액', '당기순이익', 'FCF', '총차입부채',
       '자산총계', 'EBIT_매출액', '부채비율', '차입금의존도', 'Marcap',
       'ESG등급', '지배구조', '사회', '환경'],
      dtype='object')

In [107]:
# df[df.values >30] = np.nan
# df[df.values <-30] = np.nan

In [108]:
df = df.dropna()

In [109]:
len(df)

412

In [110]:
# df_x = df[['매출액','당기순이익','FCF','총차입부채','자산총계','EBIT_매출액','부채비율','차입금의존도','Marcap','ESG등급','지배구조','사회',
#           '환경']]

In [111]:
df_x = df[['매출액','당기순이익','FCF','총차입부채','자산총계','EBIT_매출액','부채비율','Marcap','지배구조']]

In [112]:
df_y = df["Point"]

In [113]:
# train = dfa.iloc[:,5:]
# est = dfa.iloc[:,1]
# # from scipy.stats.mstats import winsorize
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(train, test,test_size=0.3)

In [114]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y,test_size=0.3)

In [115]:
from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()
rb = robust_scaler.fit(X_train)
X_train = robust_scaler.transform(X_train)
X_test = robust_scaler.transform(X_test)

In [116]:
len(y_test)

124

In [117]:
len(X_test)

124

In [118]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [119]:
y_pred = list(model.predict(X_test))
y_true = list(y_test)

In [120]:
len(y_test)

124

In [121]:
from sklearn.metrics import confusion_matrix

In [122]:
# from sklearn.metrics import classification_report
# print(classification_report(y_true, y_pred, target_names=['class 0', 'class 1']))

In [123]:
from sklearn.metrics import precision_score
precision_score(y_true, y_pred, average='weighted')

0.7903711829808507

In [124]:
confusion_matrix(y_true, y_pred, labels=[1,0])

array([[ 3, 12],
       [20, 89]])

In [125]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor

# vif = pd.DataFrame()
# vif["VIF Factor"] = [variance_inflation_factor(dfx_2015_2020.values, i) for i in range(dfx_2015_2020.shape[1])]
# vif["features"] = dfx_2015_2020.columns
# vif

In [126]:
from sklearn.preprocessing import RobustScaler
robust_scaler = RobustScaler()
robust_scaler.fit(df_x)
output = robust_scaler.transform(df_x)
output = pd.DataFrame(output, columns=df_x.columns, index=list(df_x.index.values))
output["Y"] = df_y
output.tail()

,매출액,당기순이익,FCF,총차입부채,자산총계,EBIT_매출액,부채비율,Marcap,지배구조,Y
407,2.439936,3.115229,14.361360,0.349229,1.263837,-0.795108,-0.995223,0.143290,1.0,0.0
408,-0.252921,0.145236,0.542630,-0.085921,-0.261992,0.847863,-1.798903,-0.253012,0.0,0.0
409,72.813810,-112.975257,2562.216029,13.887011,30.663646,0.593838,-0.641939,12.176668,-1.0,1.0
410,0.590107,9.203421,3.976395,-0.078441,0.252633,-0.777207,-1.755743,1.179525,2.0,0.0
411,-0.121000,-1.024553,2.069572,0.268837,0.260481,0.130448,-1.170385,0.048300,-1.0,0.0


In [127]:
# output.columns = ["X1","X2","X3","X4","X5","X6","X7","X8","X9","X10","X11","X12","X13","Y"]

In [128]:
output["Y"].value_counts()

0.0    355
1.0     57
Name: Y, dtype: int64

In [129]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_x.iloc[:,:9],df,test_size=0.3)


In [130]:
X_train = robust_scaler.fit_transform(X_train)
X_test = robust_scaler.transform(X_test)


In [131]:
model = LogisticRegression()
model.fit(X_train, y_train)


ValueError: bad input shape (288, 14)

In [ ]:
model.predict(X_test).reshape(1,-1).shape

In [ ]:
y_test.values.reshape(1,-1).shape

In [ ]:
len(X_test)

In [ ]:
model.predict(X_test).reshape(1,-1)

In [ ]:
y_test.values.reshape(1,-1)

In [ ]:
print(model.score(model.predict(X_test).reshape(1,-1), y_test.values.reshape(1,-1)))

In [ ]:
from sklearn.linear_model import LogisticRegression
model = sm.Logit.from_formula("Y ~ X1+X2+np.log(X3)+X4+np.log(X4)+X5+X6+X7+X8+np.log(X8)+X9+X10+X11+X12+X13+0", data=output)
result1 = model.fit()
print(result1.summary())

In [ ]:
from sklearn.linear_model import LinearRegression
model = sm.OLS.from_formula("Y ~ X1+X2+np.log(X3)+X4+np.log(X4)+X5+X6+X7+X8+np.log(X8)+X9+X11+X12+X13+0", data=output)
result1 = model.fit()
print(result1.summary())